In [1]:
# https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/
 
import pandas as pd

In [2]:
df = pd.read_csv("./valeursfoncieres-2021.txt", delimiter='|', decimal=',')

/tmp/ipykernel_184079/133809594.py:1: DtypeWarning: Columns (18,23,24,26,28,30,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./valeursfoncieres-2021.txt", delimiter='|', decimal=',')


In [3]:
df.columns

Index(['Code service CH', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')

In [4]:
df = df[['Valeur fonciere', 'Voie', 'Commune', 'Code departement', "Surface reelle bati", 'Nombre pieces principales', 'Surface terrain']]

In [5]:
df = df.rename(lambda c: c.lower().replace(" ", "_"), axis=1)
df

,valeur_fonciere,voie,commune,code_departement,surface_reelle_bati,nombre_pieces_principales,surface_terrain
0,185000.0,DE VOGELAS,VAL-REVERMONT,1,97.0,5.0,2410.0
1,185000.0,DE VOGELAS,VAL-REVERMONT,1,0.0,0.0,2410.0
2,10.0,ROUGEMONT,BEY,1,NaN,NaN,530.0
3,204332.0,DES ECUREUILS,BUELLAS,1,88.0,4.0,866.0
4,320000.0,DE CERTINES,MONTAGNAT,1,168.0,5.0,1426.0
...,...,...,...,...,...,...,...
3379227,383000.0,DES TOURNELLES,PARIS 04,75,30.0,2.0,NaN
3379228,260000.0,SAINTE FOY,PARIS 02,75,37.0,2.0,NaN
3379229,38000.0,DES GUILLEMITES,PARIS 04,75,0.0,0.0,NaN
3379230,38000.0,DU PETIT MUSC,PARIS 04,75,0.0,0.0,NaN


In [6]:
df = df.dropna()

In [7]:
df_sample = df.sample(n=10000, random_state=1)


In [8]:
df_sample.describe()

,valeur_fonciere,surface_reelle_bati,nombre_pieces_principales,surface_terrain
count,1.000000e+04,10000.000000,10000.000000,10000.000000
mean,5.616851e+06,99.313100,2.543000,1217.235400
std,3.126294e+07,513.307748,2.292134,4110.514839
min,5.000000e-01,0.000000,0.000000,0.000000
25%,1.250000e+05,0.000000,0.000000,266.000000
50%,2.200000e+05,70.000000,3.000000,521.500000
75%,3.900000e+05,105.000000,4.000000,1000.000000
max,2.206233e+08,21636.000000,17.000000,137288.000000


In [9]:
df = df.astype({'valeur_fonciere': float})

In [10]:
from sqlalchemy import create_engine    

engine = create_engine('sqlite://', echo=False)
df_sample.to_sql(name="fonciere", con=engine, index=False)  # reset_index() is needed to preserve index column in dumped data

with engine.connect() as conn:
    with open("fonciere.sql", "w") as f:
        for line in conn.connection.iterdump():
            f.write(line)
            f.write('\n')

In [12]:
df[df["commune"].str.startswith("PARIS ")]

,valeur_fonciere,voie,commune,code_departement,surface_reelle_bati,nombre_pieces_principales,surface_terrain
2092495,560000.0,DE MOSCOU,LE TOUQUET PARIS PLAGE,62,114.0,6.0,175.0
2092498,1800000.0,ARSENE BICAL,LE TOUQUET PARIS PLAGE,62,150.0,5.0,1001.0
2092499,1800000.0,ARSENE BICAL,LE TOUQUET PARIS PLAGE,62,150.0,5.0,750.0
2092522,602000.0,DU GAL DE GAULLE,LE TOUQUET PARIS PLAGE,62,150.0,7.0,475.0
2092523,602000.0,DU GAL DE GAULLE,LE TOUQUET PARIS PLAGE,62,150.0,7.0,1025.0
...,...,...,...,...,...,...,...
3379201,2000000.0,DU PONCEAU,PARIS 02,75,41.0,0.0,77.0
3379202,2000000.0,DU PONCEAU,PARIS 02,75,39.0,2.0,77.0
3379203,2000000.0,DU PONCEAU,PARIS 02,75,39.0,2.0,77.0
3379204,2000000.0,DU PONCEAU,PARIS 02,75,39.0,2.0,77.0
